# Agentic fighters

## Exercise 4 - Architectures: hands-on

Ahora es vuestro turno!

Vamos a dejar un buen rato para que podamos aplicar lo que hemos aprendido sobre LangGraph e intentar diseñar algunas de las arquitecturas que pueden ser más útiles. Vamos a por ello!

_(Cómo podrías utilizar estas estructuras para hacer una app de Agentic Fighters?)_

### Augmented LLM

![Augmented LLM](./../images/Augmented_LLM.png)

- Cuándo utilizar esta arquitectura?

- Algunos ejemplos donde puede ser útil:

-----

### Prompt chaining

![Prompt chaining](./../images/Prompt_chaining.png)

- Cuándo utilizar esta arquitectura?

- Algunos ejemplos donde puede ser útil:

-----

### Routing

![Routing](./../images/Routing.png)

- Cuándo utilizar esta arquitectura?

- Algunos ejemplos donde puede ser útil:

-----

### Parallelization

![Parallelization](./../images/Parallelization.png)

- Cuándo utilizar esta arquitectura?

- Algunos ejemplos donde puede ser útil:

-----

### Orchestrator-workers

![Orchestrator-workers](./../images/Orchestrator-workers.png)

- Cuándo utilizar esta arquitectura?

- Algunos ejemplos donde puede ser útil:

-----

### Evaluator-optimizer

![Evaluator-optimizer](./../images/Evaluator-optimizer.png)

- Cuándo utilizar esta arquitectura?

- Algunos ejemplos donde puede ser útil: